# Parámetros

In [1]:
BASE_DIR = '/Users/efraflores/Desktop/EF/Corner/Requests'
FILE_NAME = 'orderproduct_freshmarket.csv'

# Código

In [2]:
from pathlib import Path
from pandas import DataFrame, read_csv

class TopProducts:
    def __init__(self, base_dir: str, file_name: str) -> None:
        self.base_dir = Path(base_dir)
        self.file_name = file_name
        self.just_name = ''.join(self.file_name.split('.')[:-1])
        self.file_path = self.base_dir.joinpath(file_name)
        if not self.file_path.is_file:
            print(f'Debería haber un archivo llamado "{self.file_name}" en el directorio:\n{self.base_dir}\nAgrégalo e intenta de nuevo!')

    def get_file(self, in_chunks: bool=False, chunksize: int=10000) -> None:
        if in_chunks:
            self.df = DataFrame()
            data = read_csv(self.file_path, chunksize=chunksize)
            for i,x in enumerate(data):
                if i>2: break
                else: self.df = self.df.append(x, ignore_index=True)
        else:
            self.df = read_csv(self.file_path)
    
    def transform(self, cumsum_threshold: float, order_col: str='order_id', store_col: str='store', cols_to_group: list=['store','category','product','SKU','product_id'], cols_to_sum: list=['sales','qty_requested','qty_found']) -> None:
        self.get_file()
        self.df = self.df.groupby(cols_to_group).agg({**{order_col:'nunique'}, **{x:sum for x in cols_to_sum}}).reset_index()
        self.df.sort_values([store_col, order_col], ascending=[True, False], inplace=True)
        self.df['cumsum']=self.df[order_col]/self.df.groupby(store_col)[order_col].transform('sum')
        self.df['cumsum'] = self.df.groupby(store_col)['cumsum'].cumsum()
        self.df = self.df[self.df['cumsum']<=cumsum_threshold].reset_index(drop=True)
        self.df.rename({order_col:'orders'}, axis=1, inplace=True)
        self.df.to_csv(self.base_dir.joinpath(f'{self.just_name}_top{cumsum_threshold}.csv'), index=False, sep='\t', encoding='utf-16')

# Transformar

In [3]:
tp = TopProducts(BASE_DIR, FILE_NAME)
tp.transform(cumsum_threshold=0.8)
tp.df.head(7)

,store,category,product,SKU,product_id,orders,sales,qty_requested,qty_found,cumsum
0,1452|Fresh Market,967|Higiene oral,Pasta dental Total 12 2 x 100 ml,382646,275288,28,2226.4,36.0,36.0,0.081633
1,1452|Fresh Market,967|Higiene oral,Pasta dental Total 12 clean mint 75 ml,440895,145445,10,383.9,11.0,11.0,0.110787
2,1452|Fresh Market,1500|Cuidado cabello,Shampoo daily detox hierbas verdes Botella 700 ml,443336,321996,9,813.7,13.0,13.0,0.137026
3,1452|Fresh Market,974|Cuidado femenino,Toallas femeninas delgada con alas 14 piezas,307996,338815,9,319.0,10.0,10.0,0.163265
4,1452|Fresh Market,967|Higiene oral,Crema dental para la sensibilidad blanqueador ...,182534,337758,8,799.2,8.0,8.0,0.186589
5,1452|Fresh Market,967|Higiene oral,Pasta dental Total 12 encías reforzadas 75 ml,446644,212750,7,244.3,7.0,7.0,0.206997
6,1452|Fresh Market,967|Higiene oral,Pasta dental antiplaca y blanqueamiento 99 ml,438857,1448394,7,589.2,8.0,8.0,0.227405
